In [1]:
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.utils.np_utils import to_categorical
import _pickle as cPickle 
import pandas as pd

from keras.models import load_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
body = pd.read_csv("data/train_bodies.csv")
stances = pd.read_csv("data/train_stances.csv")
data = pd.merge(body, stances, how='right', on='Body ID')
targets = ['agree', 'disagree', 'discuss', 'unrelated']
targets_dict = dict(zip(targets, range(len(targets))))
# data['target'] = data['Stance']
data['target'] = data['Stance'].replace(targets,targets_dict.values())
data_y = data['target'].values

In [3]:
data.head()

,Body ID,articleBody,Headline,Stance,target
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,3
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,3
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,3
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,3
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,3


In [4]:
train = data
train.head()

,Body ID,articleBody,Headline,Stance,target
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,3
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,3
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,3
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,3
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,3


In [ ]:
# from gensim.scripts.glove2word2vec import glove2word2vec
# from gensim.models.keyedvectors import KeyedVectors
# glove2word2vec(glove_input_file="glove.6B.50d.txt", word2vec_output_file="gensim_glove_vectors.txt")
# glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# stop_words = set(stopwords.words('english'))
# def process(x):
#     word_tokens = word_tokenize(x)
#     embedding = [glove_model[word.lower()] for word in word_tokens if word not in stop_words and word.lower() in glove_model.wv.index2word] 
#     return embedding

In [ ]:
# train['w2v'] = (train['Headline'] + '. ' + train['articleBody']).map(process)
# train.head()

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
text = train['Headline'] + '. ' + train['articleBody']
# word_tokens = [word_tokenize(i) for i in text] 
bow_vectorizer = CountVectorizer(max_features=500, stop_words=stop_words)
bow = bow_vectorizer.fit_transform(text)  # Train set only
tfreq_vectorizer = TfidfTransformer(use_idf=False).fit(bow)
tfreq = tfreq_vectorizer.transform(bow).toarray()  # Train set only
tfidf_vectorizer = TfidfVectorizer(max_features=50, stop_words=stop_words).fit_transform(text)  # Train and test sets


In [6]:
train['bow'] = pd.Series(bow.toarray().tolist())
train['tfrep'] = pd.Series(tfreq.tolist())
train['tfidf'] = pd.Series(tfidf_vectorizer.toarray().tolist())

In [7]:
train['bow'] = train['bow'].map(lambda x: np.array(x))
train['tfrep'] = train['tfrep'].map(lambda x: np.array(x))
train['tfidf'] = train['tfidf'].map(lambda x: np.array(x))

In [8]:
train.head()

,Body ID,articleBody,Headline,Stance,target,bow,tfrep,tfidf
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1512590758911..."
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1501261740609..."
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1496395394812..."
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1512590758911..."
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1512590758911..."


In [51]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    best_score, _ = score_submission(actual,actual)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score


In [14]:
# tree model
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# model_xgb = xgb.XGBClassifier(objective='multi:softmax',learning_rate = 0.1, max_depth = 10, alpha = 10, n_estimators = 300)
# X = train['tfrep'].values
# X = np.array([np.array(i) for i in X])
# y = train.target
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model_xgb.fit(X_train,y_train)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
# model_xgb.score(X_test,y_test)

0.9194597298649325

In [49]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
lgclf = LGBMClassifier(boosting_type='gbdt', n_estimators=5000, num_leaves=8, max_depth=30,learning_rate=0.1, subsample=0.6, colsample_bytree=0.6, objective='binary')
X = train['tfrep'].values
X = np.array([np.array(i) for i in X])
y = train.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lgclf.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.6,
        importance_type='split', learning_rate=0.1, max_depth=30,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=5000, n_jobs=-1, num_leaves=8, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.6, subsample_for_bin=200000, subsample_freq=0)

In [50]:
lgclf.score(X_test,y_test)

0.9311655827913957

In [52]:
actual = y_train
predicted = lgclf.predict(X_train) 
report_score([LABELS[e] for e in actual],[LABELS[e] for e in predicted])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |   2824    |    18     |    18     |    39     |
-------------------------------------------------------------
| disagree  |    28     |    656    |     7     |     7     |
-------------------------------------------------------------
|  discuss  |    34     |    21     |   7003    |    35     |
-------------------------------------------------------------
| unrelated |     7     |     1     |    23     |   29256   |
-------------------------------------------------------------
Score: 17828.5 out of 18011.75	(98.98260857495801%)


98.98260857495801

In [53]:
actual = y_test
predicted = lgclf.predict(X_test) 
report_score([LABELS[e] for e in actual],[LABELS[e] for e in predicted])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    528    |    37     |    99     |    115    |
-------------------------------------------------------------
| disagree  |    34     |    69     |    20     |    19     |
-------------------------------------------------------------
|  discuss  |    95     |    21     |   1550    |    150    |
-------------------------------------------------------------
| unrelated |    43     |     1     |    54     |   7160    |
-------------------------------------------------------------
Score: 4013.5 out of 4551.5	(88.17972097110842%)


88.17972097110842